In [1]:
import os 
import time
import requests
import json
import re
import pandas as pd
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import pickle
from IPython.display import Image
from IPython.core.display import display, HTML

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

indexnum=0

#list of [片名，路径]
namelist=[]

#已下载无字幕番号合集
nonsublist=[]


privatelist=load_obj('privatelist')
sosolist=load_obj('sosolist')
badlist=load_obj('badlist')
collectedlist=load_obj('collectedlist')


def sortFunc(e):
    return e[1]

def get_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return (total_size/1024/1024)



def largets_file(start_path = '.'):
    largest=''
    max_size=0
    for f in next( os.walk(start_path) )[2]:
        fp = os.path.join(start_path, f)
        size=os.path.getsize(fp)
        if size>max_size:
            max_size=size
            largest=fp
    return largest

def largets_file_size(start_path = '.'):
    #print(start_path)
    max_size=0
    if len(next( os.walk(start_path) )[2])>0:
        for f in next( os.walk(start_path) )[2]:
            fp = os.path.join(start_path, f)
            size=os.path.getsize(fp)
            if size>max_size:
                max_size=size
    return max_size/1024/1024


headers = {'User-Agent':
               'Mozilla/5.0 (Windows NT 10.0; WOW64)\
               AppleWebKit/537.36(KHTML, like Gecko)\
               Chrome/55.0.2883.87 Safari/537.36'}

def javbusPage(code):
    url = 'https://www.javbus.com/'+code

    web = requests.get(url, headers=headers)
    web.encoding = 'utl-8'

    Page = web.text
    return Page
        
def imageUrl(Page):
    imageurl=re.findall(r'<a class="bigImage" href="(.*?)">',Page)
    if len(imageurl)>0:
        imageurl=imageurl[0]
    else:
        imageurl=''
    return imageurl

def actresses(Page):
    actresslist=re.findall(r'<div class="star-name"><a href="https://www.javbus.com/star/(.*?)" title=".*?">(.*?)<', Page)
    return actresslist

def existedAdress(code):
    existlist=[]
    for folder in namelist:
        if folder[0]==code:
            existlist.append(folder[1])
    return existlist

def completeinfo(code):

    Page = javbusPage(code)
    actresslist=actresses(Page)
    imageurl=imageUrl(Page)
    
    existlist=existedAdress(code)
    
    return [code,actresslist,existlist,imageurl]

def prt(msg):
    print(msg)
    print(msg,file=f)
    
def showAsLink(url,linktext,text=''):
    display(HTML(r'<p><a  href="'+url+'">'+linktext+'</a>'+text+'</p>'))
    
def linkFormat(url,linktext):
    return r'<a  href="'+url+'">'+linktext+'</a>'

def colorFormat(text,color):
    return '<font color="'+color+'">'+text+'</font>'

def wrapFormat(fmat,content):
    return '<'+fmat+'>'+content+'</'+fmat+'>'

def printHtml(contentlist):
    display(HTML(''.join(contentlist)))
    


def findmag(prefixnum):
    prefix=prefixnum[0]
    num=prefixnum[1]
    result=[]
    print(prefix.upper()+'-'+num+'开始搜索:')
    print()
    maglist=[]
    keywordlist=[]
    
    keywordlist.append(prefix+'-'+num+'r')
    keywordlist.append(prefix+'-'+num+'c')
    keywordlist.append(prefix+num+'c')
    keywordlist.append(prefix+num+'cmp4')
    
    for keyword in keywordlist:
        url = 'https://m.zhongziso.com/list/'+keyword+'/1'

        web = requests.get(url, headers=headers)
        web.encoding = 'utl-8'

        Page = web.text

        indexr=Page.find('<span class="highlight">'+keyword+'</span>')

        if (indexr!=-1):
            print(keyword+'有带字幕链接')
            result.append(keyword+'有带字幕链接')
            result.append('网址：')
            result.append(url)
            result.append('磁力链接：')

            maglist=re.findall('magnet:/?.*?(?=")', Page)
            for item in maglist:
                result.append(item)
            result.append(' ')
    
    print('-------------------------------------------------------------------------')
    print()
    return result

def findmagpure(prefixnum):
    prefix=prefixnum[0]
    num=prefixnum[1]
    result=[]
    print(prefix.upper()+'-'+num+'开始搜索:')
    print()
    maglist=[]
    keywordlist=[]
    
    keywordlist.append(prefix+'-'+num+'r')
    keywordlist.append(prefix+'-'+num+'c')
    keywordlist.append(prefix+num+'c')
    keywordlist.append(prefix+num+'cmp4')
    
    for keyword in keywordlist:
        url = 'https://m.zhongziso.com/list/'+keyword+'/1'

        web = requests.get(url, headers=headers)
        web.encoding = 'utl-8'

        Page = web.text

        indexr=Page.find('<span class="highlight">'+keyword+'</span>')

        if (indexr!=-1):
            maglist=re.findall('magnet:/?.*?(?=")', Page)
            for item in maglist:
                print(item)
                result.append(item)

    return result

def getCodes(drivelist):
    codelist=[]
    for g in drivelist:
        for path,dir_list,file_list in g:  
            for dir_name in dir_list:
                index_=dir_name.find('-')
                if (index_!=-1) and (index_<len(dir_name)-1):
                    indexnum=index_+1
                    while (dir_name[indexnum].isdigit()) and (indexnum<len(dir_name)):
                        indexnum+=1
                        if indexnum==len(dir_name):
                            break
                    code=dir_name[:indexnum]
                    codelist.append(code)
    codelist.sort()
    return codelist

# 番号信息

In [2]:
try:
    namelist=load_obj('namelist')

    #需找演员的番号列表

    drivelist=[]
    drivelist.append(os.walk(r"g:/renamed"))
    #drivelist.append(os.walk(r"g:/BEST"))
    #drivelist.append(os.walk(r"g:/Checked"))
    #drivelist.append(os.walk(r"g:/Collected"))
    #drivelist.append(os.walk(r"i:/Checked (Non Subbed)/"))

    codelist=getCodes(drivelist)

    
    pool = ThreadPool()
    results=pool.map(completeinfo,codelist[:10])
    
    
    f = open('renamedinfo.txt','w',encoding='utf-8')
    for result in results:
        #番号带链接
        printHtml(['<h1>',
                   linkFormat('https://www.javbus.com/'+result[0],result[0]),'  ',
                   '<small>',linkFormat('http://www.javlibrary.com/cn/vl_searchbyid.php?keyword='+result[0],'JavLibrary'),'</small>'
                   
        ])
        #已有文件位置
        for folder in result[2]:
            dirlink=r'localexplorer:'+folder[0].replace('\\','/')
            printHtml([
                '<big>',linkFormat(dirlink,'file://'),folder[0].replace('\\','/'),'</big>',
                '<br>',str(os.listdir(folder[0])),'   ',
                folder[1]
                
             ])
        #女优信息
        if (len(result[1])>0):
            prt('- - - - - - - - - - - - - - - - - - - - - - - - - ')
            for code,name in result[1]:
                actressurl='https://www.javbus.com/star/'+code
                isBest=0
                
                for collectedname in collectedlist:
                    if collectedname.find(name)!=-1:
                        isBest=1
                        printHtml([linkFormat(actressurl,name),'   Best','<br>','file://G:/Collected/'+collectedname])
                if isBest==0:
                    text=''
                    if name in privatelist:
                        text='Private'
                    elif name in sosolist:
                        text='Soso'
                    elif name in badlist:
                        text='Bad'
                    else:
                        text=colorFormat('PLEASE CLASSIFY','red')
                    printHtml([linkFormat(actressurl,name),'   ',text])
        #封面
        display(HTML('<img src='+result[3]+'>'))


        prt('')
        prt('')
        prt('')
        prt('===================================================================================================================')
        prt('###################################################################################################################')
        prt('===================================================================================================================')

except BaseException:
    import sys
    print(sys.exc_info()[0])
    import traceback
    print(traceback.format_exc())
finally:
    print("Press Enter to continue ...")
    #input()




###################################################################################################################


- - - - - - - - - - - - - - - - - - - - - - - - - 





###################################################################################################################


- - - - - - - - - - - - - - - - - - - - - - - - - 





###################################################################################################################


...




###################################################################################################################


- - - - - - - - - - - - - - - - - - - - - - - - - 





###################################################################################################################


- - - - - - - - - - - - - - - - - - - - - - - - - 





###################################################################################################################
Press Enter to continue ...


# 找女优作品

In [101]:
try:
    namelist=load_obj('namelist')

    #需找演员的番号列表

    drivelist=[]
    #drivelist.append(os.walk(r"g:/renamed"))
    #drivelist.append(os.walk(r"g:/BEST"))
    #drivelist.append(os.walk(r"g:/Checked"))
    drivelist.append(os.walk(r"g:/DELETED"))
    #drivelist.append(os.walk(r"f:/Checked/"))

    codelist=getCodes(drivelist)

    
    pool = ThreadPool()
    results=pool.map(completeinfo,codelist)
    
    
        
#         #番号带链接
#         codeTitleHtml=r'<h2><a  href="'+'https://www.javbus.com/'+result[0]+'">'+result[0] +'</a></h2>'
#         javLibraryLink=r'<a  href="'+'http://www.javlibrary.com/cn/vl_searchbyid.php?keyword='+result[0]+'">'+'JavLibrary' +'</a>'
#         display(HTML(codeTitleHtml+javLibraryLink))
#         #已有文件位置
#         for folder in result[2]:
#             prt(r'file://'+folder[0].replace('\\','/'))
#             dirlink=r'localexplorer:'+folder[0].replace('\\','/')
#             showAsLink(dirlink,str(os.listdir(folder[0]))+folder[1])
#         #女优信息
#         if (len(result[1])>0):
#             prt('- - - - - - - - - - - - - - - - - - - - - - - - - ')
#             for code,name in result[1]:
#                 actressurl='https://www.javbus.com/star/'+code
#                 isBest=0
#                 tag=''
#                 for collectedname in collectedlist:
#                     if collectedname.find(name)!=-1:
#                         isBest=1
#                         tag='Best'
#                         showAsLink(actressurl,name+' '+tag)
#                         #display(HTML(r'<a  href="'+actressurl+'">'+name+' '+tag+'</a>'))
#                         prt('file://G:/Collected/'+collectedname)
#                         #dirlink=r'<a  href="localexplorer:G:/Collected/'+collectedname+'">'+name+'</a>'
#                         #display(HTML(dirlink))
#                 if isBest==0:
#                     if name in privatelist:
#                         tag='Private'
#                     elif name in sosolist:
#                         tag='Soso'
#                     elif name in badlist:
#                         tag='Bad'
#                     else:
#                         tag='PLEASE CLASSIFY'
#                     showAsLink(actressurl,name+' '+tag)
#         #封面
#         display(HTML('<img src='+result[3]+'>'))


#         prt('')
#         prt('')
#         prt('')
#         prt('===================================================================================================================')
#         prt('###################################################################################################################')
#         prt('===================================================================================================================')

except BaseException:
    import sys
    print(sys.exc_info()[0])
    import traceback
    print(traceback.format_exc())
finally:
    print("Press Enter to continue ...")
    #input()

Press Enter to continue ...


In [103]:
for result in results:
        for code,name in result[1]:
            if name=='篠田ゆう':
                if len(result[1])==1:
                    codeTitleHtml=r'<h2><a  href="'+'https://www.javbus.com/'+result[0]+'">'+result[0] +'</a></h2>'
                    javLibraryLink=r'<a  href="'+'http://www.javlibrary.com/cn/vl_searchbyid.php?keyword='+result[0]+'">'+'JavLibrary' +'</a>'
                    display(HTML(codeTitleHtml+javLibraryLink))
                    display(HTML('<img src='+result[3]+'>'))
                    path=result[2][0][0]
                    dst=path.replace('DELETED','3')
                    try:
                        os.rename(path,dst)
                    except:
                        None
                        


In [104]:
for result in results:
        for code,name in result[1]:
             if name=='篠田ゆう':
#                 if len(result[1])==1:
#                     path=result[2][0][0]
#                     dst=path.replace('Checked','3')
#                     try:
#                         os.rename(path,dst)
#                     except:
#                         None
                try:
                    print(result[1])
                    #番号带链接
                    codeTitleHtml=r'<h2><a  href="'+'https://www.javbus.com/'+result[0]+'">'+result[0] +'</a></h2>'
                    javLibraryLink=r'<a  href="'+'http://www.javlibrary.com/cn/vl_searchbyid.php?keyword='+result[0]+'">'+'JavLibrary' +'</a>'
                    display(HTML(codeTitleHtml+javLibraryLink))
                    #已有文件位置
                    for folder in result[2]:
                        prt(r'file://'+folder[0].replace('\\','/'))
                        dirlink=r'localexplorer:'+folder[0].replace('\\','/')
                        showAsLink(dirlink,str(os.listdir(folder[0]))+folder[1])
                    display(HTML('<img src='+result[3]+'>'))
                    prt('===================================================================================================================')
                except:
                    None

[('2pv', '篠田ゆう'), ('2wr', '春原未来'), ('7hw', '夏目優希'), ('81j', '蓮実クレア'), ('83i', '松本メイ'), ('p2t', '牧村ひな')]


file://g:/DELETED/LZPL-008


[('lvc', '内村りな'), ('8xb', 'HIKARI'), ('6v9', '藤原ひとみ'), ('2yu', '朝倉ことみ'), ('2yf', '芹沢つむぎ'), ('2t4', 'AIKA'), ('2pv', '篠田ゆう'), ('2dq', '北川瞳'), ('2di', '友田彩也香')]


file://g:/DELETED/MIRD-130


[('2pv', '篠田ゆう'), ('qn5', '佐々波綾'), ('7ve', '倉多まお'), ('p8y', '佐々木あき'), ('ozw', '若菜奈央'), ('mj2', '初川みなみ'), ('92l', '水野朝陽'), ('79q', '神咲詩織'), ('2de', 'JULIA'), ('2v4', '秋山祥子'), ('4xe', '冬月かえで'), ('1fw', 'つぼみ'), ('p84', '椎名そら'), ('pcm', '香椎りあ'), ('run', '桐谷なお'), ('rl1', '星奈あい'), ('rc0', '冴木エリカ'), ('reg', '美谷朱里'), ('rb4', '森川アンナ'), ('qvv', '河南実里'), ('q44', '栄川乃亜'), ('q02', '姫川ゆうな'), ('q1c', '二階堂ゆり'), ('q1z', '宮沢ゆかり'), ('ppk', '麻里梨夏'), ('s7u', '平沢すず')]


[('2t', '風間ゆみ'), ('p6', '川上ゆう（森野雫）'), ('2pv', '篠田ゆう'), ('7xb', '尾上若葉'), ('8y0', '卯水咲流'), ('cnw', '大場ゆい'), ('n0l', '篠田あゆみ'), ('nsy', '美咲かんな'), ('om5', '藤本紫媛'), ('p8y', '佐々木あき'), ('95f', '本田莉子'), ('nrq', '香山美桜'), ('na9', '松嶋葵'), ('n81', '鈴原エミリ'), ('m54', '林ゆな'), ('lz6', '佐々木恋海（向井恋）'), ('2m3', '大槻ひびき'), ('7ve', '倉多まお'), ('2wr', '春原未来'), ('i4k', '川菜美鈴'), ('hy6', '川村まや'), ('oxb', '北嶋あん'), ('q76', '彩奈リナ（七原あかり）'), ('81j', '蓮実クレア'), ('org', '江上しほ'), ('efh', '長谷川夏樹'), ('92l', '水野朝陽'), ('b9f', 'KAORI'), ('n58', '夏希みなみ'), ('7ry', '神ユキ')]


file://g:/DELETED/TOMN-108


[('2pv', '篠田ゆう'), ('2ud', '真木今日子'), ('7et', 'あすか光希'), ('933', '夢咲りぼん'), ('93p', '椎名まりな'), ('944', '水希杏'), ('mb7', 'あゆみ翼'), ('mg9', '青葉優香'), ('nif', '御舟みこと'), ('o4r', '原波瑠（原羽瑠）'), ('85q', '彩城ゆりな'), ('91c', '椿かなり')]


file://g:/DELETED/ZUKO-077_CAVI


# 种子筛选

In [303]:
try:
    namelist=load_obj('namelist')

    #需找演员的番号列表

    drivelist=[]
    drivelist.append(os.walk(r"g:/sis"))
    #drivelist.append(os.walk(r"g:/BEST"))
    #drivelist.append(os.walk(r"g:/Checked"))
    #drivelist.append(os.walk(r"g:/Collected"))

    #drivelist.append(os.walk(r"i:/Checked (Non Subbed)/"))

    codelist=[]
    for g in drivelist:
        for path,dir_list,file_list in g:  
            for dir_name in dir_list:
                index_=dir_name.find('-')
                if (index_!=-1) and (index_<len(dir_name)-1):
                    indexnum=index_+1
                    while (dir_name[indexnum].isdigit()) and (indexnum<len(dir_name)):
                        indexnum+=1
                        if indexnum==len(dir_name):
                            break
                    code=dir_name[:indexnum]
                    codelist.append(code)
    codelist.sort()

    pool = ThreadPool()
    results=pool.map(completeinfo,codelist)
    f = open('renamedinfo.txt','w',encoding='utf-8')


except BaseException:
    import sys
    print(sys.exc_info()[0])
    import traceback
    print(traceback.format_exc())
finally:
    
    print("Press Enter to continue ...")
    #input()

Press Enter to continue ...


In [ ]:
for result in results:
    if len(result[2])>0:
        display(HTML(r'<a  href="'+'https://www.javbus.com/'+result[0]+'">'+result[0] +'</a>'))
        prt('JavLibrary for more info:')
        prt(r'http://www.javlibrary.com/cn/vl_searchbyid.php?keyword='+result[0])
        display(HTML('<img src='+result[3]+'>'))
        for folder in result[2]:
            prt(r'file://'+folder[0].replace('\\','/'))
            prt(os.listdir(folder[0]))
            dirlink=r'localexplorer:'+folder[0].replace('\\','/')
            dirlink=r'<a  href="'+dirlink+'">'+folder[0] +'</a>'
            display(HTML(dirlink))
        if (len(result[1])>0):
            prt('- - - - - - - - - - - - - - - - - - - - - - - - - ')
            for code,name in result[1]:
                prt(name)
                actressurl='https://www.javbus.com/star/'+code
                for collectedname in collectedlist:
                    if collectedname.find(name)!=-1:
                        prt('file://G:/Collected/'+collectedname)
                        #dirlink=r'<a  href="localexplorer:G:/Collected/'+collectedname+'">'+name+'</a>'
                        #display(HTML(dirlink))
                if name in privatelist:
                    display(HTML(r'<a  href="'+actressurl+'">'+'PRIVATE'+'</a>'))
                elif name in sosolist:
                    display(HTML(r'<a  href="'+actressurl+'">'+'SOSO'+'</a>'))
                elif name in badlist:
                    display(HTML(r'<a  href="'+actressurl+'">'+'BAD'+'</a>'))
                else:
                    display(HTML(r'<a  href="'+actressurl+'">'+'PLEASE CLASSIFY'+'</a>'))


        prt('-------------------------------------------------------------------------')

In [ ]:
namelist=load_obj('namelist')

for x in namelist:
    print(x)

# Collected文件夹校验

In [24]:
drivelist=[]

root=r"g:/Collected"
for dir_name in next( os.walk(root) )[1]:
    drivelist.append(dir_name)

for actressname in drivelist:
    print('------------------------------------------')
    print(actressname)
    path=os.path.join(root, actressname)
    g=os.walk(path)
    codelist=[]
    for path,dir_list,file_list in g:  
        
        for dir_name in dir_list:
            index_=dir_name.find('-')
            if (index_!=-1) and (index_<len(dir_name)-1):
                indexnum=index_+1
                while (dir_name[indexnum].isdigit()) and (indexnum<len(dir_name)):
                    indexnum+=1
                    if indexnum==len(dir_name):
                        break
                code=dir_name[:indexnum]

                codelist.append(code)

    codelist.sort()

    pool = ThreadPool()
    results=pool.map(completeinfo,codelist)
    results.sort(key=sortFunc)
    for result in results:
        correct=0
        if len(result[1])==0:
            correct=1
        if len(result[1])==1:
            if actressname.find(result[1][0][1])!=-1:
                correct=1
        if len(result[1])>1:
            for code,name in result[1]:
                if name.find('（')!=-1:
                    name=name[:name.find('（')]
                if actressname.find(name)!=-1:
                    correct=1
                    break
        if correct==0:
            print()
            for line in result:
                print(line)

------------------------------------------
AIKA
------------------------------------------
JULIA
------------------------------------------
NAOMI
------------------------------------------
RION（宇都宮しをん）
------------------------------------------
sex_friend
------------------------------------------
あおいれな（葵玲奈）
------------------------------------------
あべみかこ（安部未华子）
------------------------------------------
あやみ旬果
------------------------------------------
さくらゆら（樱由罗）
------------------------------------------
ちとせりこ_母乳
------------------------------------------
ななせ麻衣（菅野紗世）（南田みさき）
------------------------------------------
ほしの景子
------------------------------------------
七沢みあ
------------------------------------------
七瀬ルナ
------------------------------------------
三上悠亜
------------------------------------------
三田杏
------------------------------------------
上原瑞穂
------------------------------------------
並木優
------------------------------------------
乙都さきの
---------------------------------

------------------------------------------
蒼星ユリ
------------------------------------------
藤川れいな
------------------------------------------
西内るな
------------------------------------------
西宮ゆめ
------------------------------------------
西田カリナ
------------------------------------------
谷田部和沙
------------------------------------------
跡美しゅり

WANZ-280
[('m09', '裕木まゆ'), ('1fw', 'つぼみ')]
[]
https://pics.javbus.com/cover/4l21_b.jpg

MUM-216
[('pcx', '森星いまり'), ('pne', '琴沖華凛')]
[]
https://pics.javbus.com/cover/5ck0_b.jpg
------------------------------------------
輝月あんり
------------------------------------------
辻本杏
------------------------------------------
金澤セレナ
------------------------------------------
鈴村あいり
------------------------------------------
長瀬麻美
------------------------------------------
長谷川るい
------------------------------------------
長谷川リホ_早期
------------------------------------------
関根奈美
------------------------------------------
青井いちご

BUR-427
[('ilh', 'のぞみ'), ('8tr', '平野あおい'),

# 自动归类

# 杂项